In [ ]:
## Package used to import Code from Google Coolab to Word
!pip install pandoc
import pandoc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.3 MB/s eta 0:00:00
  Created wheel for pandoc: filename=pandoc-2.3-py3-none-any.whl size=33262 sha256=a9aadfbd2114e92e8fa65f53592e1932650f31e1d6de73b097588f5ecc4e5f33
  Stored in directory: /root/.cache/pip/wheels/76/27/c2/c26175310aadcb8741b77657a1bb49c50cc7d4cdbf9eee0005
Successfully built pandoc


In [ ]:
# Importing All librabries to be used
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import tensorflow
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Conv1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
#from sklearn.cross_validation import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Conv1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import numpy as np
import pandas as pd
#from sklearn.kernel_approximation import RBFSampler
#from sklearn.linear_model import SGDClassifier
#from sklearn.cross_validation import train_test_split
#from sklearn import svm
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import cycle
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import auc,roc_curve,roc_auc_score

In [ ]:
## multi-class precision,recall, and accuracy
def accuracy(confusion_matrix):
    diagonal_sum = confusion_matrix.trace()
    sum_of_all_elements = confusion_matrix.sum()
    return diagonal_sum / sum_of_all_elements

def precision(label, confusion_matrix):
    col = confusion_matrix[:, label]
    return confusion_matrix[label, label] / col.sum()

def recall(label, confusion_matrix):
    row = confusion_matrix[label, :]
    return confusion_matrix[label, label] / row.sum()

def precision_macro_average(confusion_matrix):
    rows, columns = confusion_matrix.shape
    sum_of_precisions = 0
    for label in range(rows):
        sum_of_precisions += precision(label, confusion_matrix)
    return sum_of_precisions / rows

def recall_macro_average(confusion_matrix):
    rows, columns = confusion_matrix.shape
    sum_of_recalls = 0
    for label in range(columns):
        sum_of_recalls += recall(label, confusion_matrix)
    return sum_of_recalls / columns

## draw roc curve

In [ ]:
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)


    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(6):
        fpr[i], tpr[i], _ = metrics.roc_curve(y_test[:,i].astype(int), y_pred[:,i])

        roc_auc[i] = auc(fpr[i], tpr[i])
# plot model roc curve

    plt.figure(figsize=(6, 6))
    lw = 2

    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    target_classes = ['U2R','BFA','DDoS','DoS', 'Probe ','Normal']

    plt.title('Receiver Operating Characteristic(ROC) Curve')
    plt.legend(loc="lower right")
    colors = cycle(['blue', 'red', 'green', 'yellow','orange','black'])

    for i, color in zip(range(6), colors):
        plt.plot(fpr[i], tpr[i], color=color, lw=lw, label= target_classes[i] + " "' (' 'AUC = {1:0.2f} )'''.format(i, roc_auc[i]))

        plt.plot([0, 1], [0, 1], color='black', lw=lw, linestyle='--')

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.legend(loc = 'lower right')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')


    return roc_auc_score(y_test, y_pred, average=average)

In [ ]:
#from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error, roc_curve, classification_report,auc)

traindata = pd.read_csv('../input/ransomware/Charlotte_metasploitable-2_70.csv', header=None)
testdata = pd.read_csv('../input/ransomware/Charlotte_metasploitable-2_30.csv', header=None)
## new convesion



from sklearn.preprocessing import LabelEncoder

#print(traindata.columns[0:26])
for column in traindata.columns:
    if traindata[column].dtype == type(object):
        le = LabelEncoder()
        traindata[column] = le.fit_transform(traindata[column])


for column in testdata.columns:
    if testdata[column].dtype == type(object):
        le = LabelEncoder()
        testdata[column] = le.fit_transform(testdata[column])

X = traindata.iloc[:,0:83]
Y = traindata.iloc[:,83:84]


C = testdata.iloc[:,83:84]
T = testdata.iloc[:,0:83]

scaler = Normalizer().fit(X)
trainX = scaler.transform(X)

scaler = Normalizer().fit(T)
testT = scaler.transform(T)


# traindata = np.array(trainX)
# trainlabel = np.array(Y)

# testdata = np.array(testT)
# testlabel = np.array(C)
y_train1 = np.array(Y)

y_test1 = np.array(C)
#
y_train= to_categorical(y_train1)
y_test= to_categorical(y_test1)

#y_train= y_train1
#y_test= y_test1



# #
# ### reshape input to be [samples, time steps, features]
X_train = np.reshape(trainX, (trainX.shape[0],trainX.shape[1],1))
X_test = np.reshape(testT, (testT.shape[0],testT.shape[1],1))

### Calling the cnn sequential
cnn = Sequential()
cnn.add(Conv1D(64, 3, padding="same",strides=1,activation="relu",input_shape=(83, 1)))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(512, activation="relu"))#128
cnn.add(Dense(256, activation="relu"))
cnn.add(Dropout(0.5))
cnn.add(Dense(6, activation="softmax"))

# define optimizer and objective, compile cnn

cnn.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])

### Result oriented
result= cnn.fit(X_train, y_train,
                #validation_split=0.30
                validation_data=(X_test,y_test )
                ,epochs=20, batch_size=32)

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# make predictions
expected = y_test

#cm = metrics.confusion_matrix(expected, predicted)


cm = confusion_matrix(expected.argmax(axis=1), y_pred.argmax(axis=1))

print('Confusion Matrix')

print(cm)

## Classification Report
print('Classification Report')

print(classification_report(expected, y_pred))


In [ ]:
!jupyter nbconvert --to html /Charlotte/Google_colab .ipynb

[NbConvertApp] WARNING | pattern '/Charlotte/Google_colab' matched no files
[NbConvertApp] WARNING | pattern '.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting